# Introduction

In this ipython notebook I will implement Linear Regression and its various variations and optimized variations from scratch. To understand the implementation I will also plot the optimizations process in cases of Gradient descent. In this notebook we will implement each algorithm with its own class and then compare performance with sklearn's implementation of algorithm. <br>
To use and take advantage of sklearn's consisten and easy to use interface, I will creating our custom implementation using sklearn's base classes **BaseEstimator**, **RegressionMixin**.
<br>
### Linear Regression (Ordinary Least Squares)
* Simple Linear Regression 
* Multiple Linear Regression
<br>

### Regularized Variations
To prevent overfitting in polynomial regression I will also implement with Regularization
* Lasso (L1 Norm)
* Ridge (L2 Norm)
* Elastic Net (Linear Combination of L1 and L2 norm - L1 + L2)

### Gradient Descent for model parameters
I will also implement Linear Regression with Gradient Descent Convex Optimization
* Batch Gradient Descent
* Stochastic Gradient Descent

In [1]:
import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_boston, load_diabetes
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.base import BaseEstimator, RegressorMixin

# Preprocessing

## Loading Dataset

In [111]:
boston = load_boston()
diabetes = load_diabetes()

dataset = pd.read_csv('data.csv')
X = dataset.iloc[:,0].values.reshape(-1,1)
y = dataset.iloc[:,1].values

xTrain, xTest, yTrain, yTest = train_test_split(X,y)

xScaler = StandardScaler()
xTrain = xScaler.fit_transform(xTrain)
xTest = xScaler.transform(xTest)

# Simple Linear Regression

First let's see at simple linear regression and their mathematical formulas. All the formulas presented in this or other notebooks won't be dervied, that's out of scope for these notebooks. For more explanation read full post at my blog [sahilsehwag](http://www.sahilsehwag.wordpress.com)
<br>

## $covariance(x,y) = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}{n-1}$
## $variance(x) = \frac{\sum_{i=1}^{n} (x_i - \bar{x})^2}{n-1}$
Note that we are using "m" but in most of the books or any kind of resources you will find this as referred to as $\beta$, don't get confused we are just using "m" for familiarity
### $y = mx + b$
## $m=\frac{covariance(x,y)}{variance(x)}$
### $b=\bar{y} - m\bar{x} $

Let's look at our cost function. The cost function we are using is called *Residual Sum of Squares*
### $R_{SS} = \sum_{i=1}^{n} (y_i - \hat{y}_i) = \sum_{i=1}^{n} (y_i - mx_i)$

Finally We will look at our scoring function which is called *Coefficeint of Determination*
## $r^2 = 1 - \frac{R_{ss}(\hat{y})}{R_{ss}(\bar{y})}$

In [3]:
class CustomLinearRegression(BaseEstimator, RegressorMixin):
    def __init__(self, plot=False):
        self.plot = plot
    
    def fit(self, X, y=None):
        self._check_params(X, y)
        self._simple_linear_regression(X, y)
        self.fitted_ = True
        
        return self
    
    def predict(self, X):
        if self.fitted_ == None:
            raise Exception('"predict()" called before fit()')
        else:
            ys = list()
            for x in X:
                y = self.m * x[0] + self.c
                ys.append(y)
            return ys
        
    def score(self, X, y):
        if self.fitted_ == None:
            raise Exception('"score()" called before fit()')
        else:
            y_pred = self.predict(X)
            rss = ((y - y_pred) ** 2).sum()
            rss_mean = ((y - y.mean()) ** 2).sum()
            return 1 - (rss/rss_mean)
    
    def _simple_linear_regression(self, X, y=None):
        cov = ((X[:,0] - X[:,0].mean()) * (y - y.mean())).sum() / (len(y)-1)
        var = ((X[:,0] - X[:,0].mean()) ** 2).sum() / (len(y) - 1)
        self.m = cov/var
        self.c = y.mean() - self.m * X[:,0].mean()
        return
    
    def _check_params(self, X, y):
        if type(self.plot) != bool:
            raise Exception('"plot" should be boolean')
        

### LinearRegression vs CustomLinearRegression

In [4]:
skModel = LinearRegression()
custModel = CustomLinearRegression()

skModel.fit(xTrain, yTrain)
custModel.fit(xTrain, yTrain)

print(cross_val_score(estimator=skModel, X=xTest, y=yTest, cv=3))
print(cross_val_score(estimator=custModel, X=xTest, y=yTest, cv=3))

[ 0.02554728  0.4368973   0.20923646]
[ 0.02554728  0.4368973   0.20923646]


# Multiple Linear Regression

Multiple Linear Regression is just a generalization of Simple Linear Regression. It is a generalization for n dimensions.

## $ y = b + \sum_{i=1}^{n} m_ix_i $

In [5]:
class MultipleLinearRegression(BaseEstimator, RegressorMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self._check_params(X, y)
        self._multiple_linear_regression(X, y)
        self.fitted_ = True
        
        return self
    
    def predict(self, X):
        if self.fitted_ == None:
            raise Exception('"predict()" called before fit()')
        else:
            ys = list()
            for x in X:
                y = self.b + (self.M * x).sum()
                ys.append(y)
            return ys
        
    def score(self, X, y):
        if self.fitted_ == None:
            raise Exception('"score()" called before fit()')
        else:
            y_pred = self.predict(X)
            rss = ((y - y_pred) ** 2).sum()
            rss_mean = ((y - y.mean()) ** 2).sum()
            return 1 - (rss/rss_mean)
    
    def _multiple_linear_regression(self, X, y=None):
        M = np.dot(np.linalg.inv(np.dot(np.transpose(X), X)), np.dot(np.transpose(X), y))
        
        b = yTest.mean()
        for i in range(X.shape[1]):
            b -= M[i] * X[:,i].mean()
        
        self.M = M
        self.b = b
        return
        
            
    def _check_params(self, X, y):
        pass

### Loading dataset with multiple features

In [6]:
dataset = boston
X = dataset.data
y = dataset.target

xTrain, xTest, yTrain, yTest = train_test_split(X,y)

### LinearRegression vs MultipleLinearRegression

In [7]:
skModel = LinearRegression()
custModel = MultipleLinearRegression()

skModel.fit(xTrain, yTrain)
custModel.fit(xTrain, yTrain)

print(cross_val_score(estimator=skModel, X=xTest, y=yTest, cv=3))
print(cross_val_score(estimator=custModel, X=xTest, y=yTest, cv=3))

[ 0.75293615  0.59266436  0.53107781]
[ 0.74489941  0.54326411  0.43325086]


# Regularization
When doing polynomial regression, higher degree terms can lead to overfitting. To solve this problem regularization is used which adds a penalty term that increases penalty as we add more higher degree terms.<br>
Mainly we do this by adding penalty to our **Residual Sum of Squares**. There are two regularization techniques which are:
* L1 Norm i.e Lasso
* L2 Norm i.e Ridge
* ElasticNet uses Linear Combination of L1 and L2

# Lasso

In previous section we mentioned Lasso uses L1 regularization. Lasso stands for **_Least Absolute Shrinkage and Selection Operator_**. <br>
$\lambda$ is called **_Shrinkage Parameter_**. It is a hyperparameter that we have to select ourselves

## $ R_{ss} = (\sum_{i=1}^{n} y_i - mx^p_i) + \lambda \sum_{j=1}^{p} m_j $

In [8]:
class CustomLasso(BaseEstimator, RegressorMixin):
    def __init__(self, alpha=1.0, tol=0.0001, max_iter=1000):
        self.alpha = alpha
        self.tol = tol
        self.max_iter = max_iter
    
    def fit(self, X, y=None):
        self._check_params(X, y)
        self._multiple_linear_regression(X, y)
        self.fitted_ = True
        
        return self
    
    def predict(self, X):
        if self.fitted_ == None:
            raise Exception('"predict()" called before fit()')
        else:
            ys = list()
            for x in X:
                y = self.b + (self.M * x).sum()
                ys.append(y)
            return ys
        
    def score(self, X, y):
        if self.fitted_ == None:
            raise Exception('"score()" called before fit()')
        else:
            y_pred = self.predict(X)
            rss = ((y - y_pred) ** 2).sum() + np.sum([(m * self.alpha) for m in self.M])
            rss_mean = ((y - y.mean()) ** 2).sum() + np.sum([(m * self.alpha) for m in self.M])
            return 1 - (rss/rss_mean)
    
    def _multiple_linear_regression(self, X, y=None):
        M = np.dot(np.linalg.inv(np.dot(np.transpose(X), X)), np.dot(np.transpose(X), y))
        
        b = yTest.mean()
        for i in range(X.shape[1]):
            b -= M[i] * X[:,i].mean()
        
        self.M = M
        self.b = b
        return
        
            
    def _check_params(self, X, y):
        if type(self.alpha) in (float, int):
            pass
        else:
            raise Exception('Invalid type of "alpha"')
            
        if type(self.tol) in (float, int):
            pass
        else:
            raise Exception('Invalid type of "tol"')
        
        if type(self.max_iter) in (float, int):
            pass
        else:
            raise Exception('Invalid type of "max_iter"')
        

### Lasso vs CustomLasso

In [9]:
skModel = Lasso()
custModel = CustomLasso()

skModel.fit(xTrain, yTrain)
custModel.fit(xTrain, yTrain)

print(cross_val_score(estimator=skModel, X=xTest, y=yTest, cv=5))
print(cross_val_score(estimator=custModel, X=xTest, y=yTest, cv=5))

[ 0.54637001  0.79931518  0.33287788  0.74097513  0.50645879]
[ 0.69778664  0.80636858  0.42816501  0.77958305  0.14928785]


# Ridge

In previous section we mentioned Ridge uses L2 regularization

## $ R_{ss} = (\sum_{i=1}^{n} y_i - mx^p_i) + \lambda \sum_{j=1}^{p} m_j^2 $

In [10]:
class CustomRidge(BaseEstimator, RegressorMixin):
    def __init__(self, alpha=1.0, tol=0.0001, max_iter=1000):
        self.alpha = alpha
        self.tol = tol
        self.max_iter = max_iter
    
    def fit(self, X, y=None):
        self._check_params(X, y)
        self._multiple_linear_regression(X, y)
        self.fitted_ = True
        
        return self
    
    def predict(self, X):
        if self.fitted_ == None:
            raise Exception('"predict()" called before fit()')
        else:
            ys = list()
            for x in X:
                y = self.b + (self.M * x).sum()
                ys.append(y)
            return ys
        
    def score(self, X, y):
        if self.fitted_ == None:
            raise Exception('"score()" called before fit()')
        else:
            y_pred = self.predict(X)
            rss = ((y - y_pred) ** 2).sum() + np.sum([(m * self.alpha) ** 2 for m in self.M])
            rss_mean = ((y - y.mean()) ** 2).sum() + np.sum([(m * self.alpha) ** 2 for m in self.M])
            return 1 - (rss/rss_mean)
    
    def _multiple_linear_regression(self, X, y=None):
        M = np.dot(np.linalg.inv(np.dot(np.transpose(X), X)), np.dot(np.transpose(X), y))
        
        b = yTest.mean()
        for i in range(X.shape[1]):
            b -= M[i] * X[:,i].mean()
        
        self.M = M
        self.b = b
        return
        
            
    def _check_params(self, X, y):
        if type(self.alpha) in (float, int):
            pass
        else:
            raise Exception('Invalid type of "alpha"')
            
        if type(self.tol) in (float, int):
            pass
        else:
            raise Exception('Invalid type of "tol"')
        
        if type(self.max_iter) in (float, int):
            pass
        else:
            raise Exception('Invalid type of "max_iter"')
        

### Ridge vs CustomRidge

In [11]:
skModel = Ridge()
custModel = CustomRidge()

skModel.fit(xTrain, yTrain)
custModel.fit(xTrain, yTrain)

print(cross_val_score(estimator=skModel, X=xTest, y=yTest, cv=5))
print(cross_val_score(estimator=custModel, X=xTest, y=yTest, cv=5))

[ 0.65866101  0.81956924  0.4712709   0.65542133  0.37941599]
[ 0.61016662  0.76204059  0.4082369   0.68379681  0.14352686]


# ElasticNet
## $ R_{ss} = (\sum_{i=1}^{n} y_i - mx^p_i) + \lambda \sum_{j=1}^{p} m_j+ \lambda \sum_{j=1}^{p} m_j^2 $

In [12]:
class CustomElasticNet(BaseEstimator, RegressorMixin):
    def __init__(self, alpha=1.0, tol=0.0001, max_iter=1000):
        self.alpha = alpha
        self.tol = tol
        self.max_iter = max_iter
    
    def fit(self, X, y=None):
        self._check_params(X, y)
        self._multiple_linear_regression(X, y)
        self.fitted_ = True
        
        return self
    
    def predict(self, X):
        if self.fitted_ == None:
            raise Exception('"predict()" called before fit()')
        else:
            ys = list()
            for x in X:
                y = self.b + (self.M * x).sum()
                ys.append(y)
            return ys
        
    def score(self, X, y):
        if self.fitted_ == None:
            raise Exception('"score()" called before fit()')
        else:
            y_pred = self.predict(X)
            rss = ((y - y_pred) ** 2).sum() + np.sum([(m * self.alpha) for m in self.M]) + np.sum([(m * self.alpha) ** 2 for m in self.M])
            rss_mean = ((y - y.mean()) ** 2).sum() + np.sum([(m * self.alpha) ** 2 for m in self.M]) + np.sum([(m * self.alpha) ** 2 for m in self.M])
            return 1 - (rss/rss_mean)
    
    def _multiple_linear_regression(self, X, y=None):
        M = np.dot(np.linalg.inv(np.dot(np.transpose(X), X)), np.dot(np.transpose(X), y))
        
        b = yTest.mean()
        for i in range(X.shape[1]):
            b -= M[i] * X[:,i].mean()
        
        self.M = M
        self.b = b
        return
        
            
    def _check_params(self, X, y):
        if type(self.alpha) in (float, int):
            pass
        else:
            raise Exception('Invalid type of "alpha"')
            
        if type(self.tol) in (float, int):
            pass
        else:
            raise Exception('Invalid type of "tol"')
        
        if type(self.max_iter) in (float, int):
            pass
        else:
            raise Exception('Invalid type of "max_iter"')
        

### ElasticNet vs CustomElasticNet

In [13]:
skModel = ElasticNet()
custModel = CustomElasticNet()

skModel.fit(xTrain, yTrain)
custModel.fit(xTrain, yTrain)

print(cross_val_score(estimator=skModel, X=xTest, y=yTest, cv=5))
print(cross_val_score(estimator=custModel, X=xTest, y=yTest, cv=5))

[ 0.57552499  0.80697554  0.37256972  0.7590873   0.51621722]
[ 0.6468721   0.76899432  0.43179247  0.71058854  0.17398641]


# SGD

In [120]:
class CustomSGDRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, alpha=0.0001, tol=0.0001, max_iter=1000, penalty='l2'):
        self.alpha = alpha
        self.tol = tol
        self.max_iter = max_iter
        self.penalty = penalty
    
    def fit(self, X, y=None):
        self._check_params(X, y)
        self._sgd(X, y)
        self.fitted_ = True
        
        return self
    
    def predict(self, X):
        if self.fitted_ == None:
            raise Exception('"predict()" called before fit()')
        else:
            ys = list()
            for x in X:
                y = self.b + (self.M * x).sum()
                ys.append(y)
            return np.array(ys)
        
    def score(self, X, y):
        if self.fitted_ == None:
            raise Exception('"score()" called before fit()')
        else:
            y_pred = self.predict(X)
            rss = self._cost(X,y,y_pred)
            rss_mean = self._cost(X,y,y.mean())
            return 1 - (rss/rss_mean)
    
    def _sgd(self, X, y=None):
        m = np.array([0 for _ in range(X.shape[1])], dtype=np.float64)
        b = 0
        
        for i in range(self.max_iter):
            m_slope = np.array([0 for _ in range(X.shape[1])], dtype=np.float64)
            b_slope = 0
            for j in range(y.shape[0]):
                xi = X[j]
                yi = y[j]
                
                m_slope += (-2/y.shape[0]) * (yi - (m * xi).sum() - b) * xi
                b_slope += (-2/y.shape[0]) * (yi - (m * xi).sum() - b)
            
            m = m - m_slope * self.alpha
            b = b - b_slope * self.alpha
            
        self.M = m
        self.b = b
        return
        
    def _cost(self, X, y, y_pred):
        if self.penalty == None:
            rss = ((y - y_pred) ** 2).sum()
        elif self.penalty == 'l1':
            rss = ((y - y_pred) ** 2).sum() + np.sum([(m * self.alpha) for m in self.M])
        elif self.penalty == 'l2':
            rss = ((y - y_pred) ** 2).sum() + np.sum([(m * self.alpha) ** 2 for m in self.M])
        elif self.penalty == 'elasticnet':
            rss = ((y - y_pred) ** 2).sum() + np.sum([(m * self.alpha) for m in self.M]) + np.sum([(m * self.alpha) ** 2 for m in self.M])
        
        return rss
            
    def _check_params(self, X, y):
        if type(self.alpha) in (float, int):
            pass
        else:
            raise Exception('Invalid type of "alpha"')
            
        if type(self.tol) in (float, int):
            pass
        else:
            raise Exception('Invalid type of "tol"')
        
        if type(self.max_iter) in (float, int):
            pass
        else:
            raise Exception('Invalid type of "max_iter"')
            
        if type(self.penalty) in (str,None) and self.penalty in ('l1', 'l2', 'elasticnet'):
            pass
        else:
            raise Exception('Invalid type of "penalty"')
        

In [121]:
dataset = boston
X = dataset.data
y = dataset.target

xTrain, xTest, yTrain, yTest = train_test_split(X,y)

xScaler = StandardScaler()
xTrain = xScaler.fit_transform(xTrain)
xTest = xScaler.transform(xTest)

### SGDRegressor vs CustomSGDRegressor

In [122]:
skModel = SGDRegressor(tol=0.0001)
custModel = CustomSGDRegressor(alpha=0.01)

skModel.fit(xTrain, yTrain)
custModel.fit(xTrain, yTrain)

CustomSGDRegressor(alpha=0.01, max_iter=1000, penalty='l2', tol=0.0001)